# Applied Data Science Capstone
## Battle of Neiborhood Location
### Created by dvh  -  Oct 2018

## Scrape the Wikipedia page

In [1]:
import io
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



### Table of postal codes

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = soup.find('table', class_='wikitable sortable')
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


### Transform the data into a pandas dataframe

In [3]:
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
df = df[df.Borough != 'Not assigned']
idx=df.index[df.Neighbourhood == 'Not assigned']
df['Neighbourhood'] = df['Borough'].where(df['Neighbourhood']=='Not assigned', df['Neighbourhood'])
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()


### Get the latitude and the longitude coordinates of each neighborhood. 

In [ ]:
url="https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
file=requests.get(url).content


In [4]:
df2=pd.read_csv('Geospatial_Coordinates.csv')
df2.columns = ['Postcode', 'Latitude','Longitude']
df1 = pd.merge(df, df2, on='Postcode')

In [90]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))
borough_Toronto_data = df1[df1['Borough'].str.contains("Toronto")].reset_index(drop=True)
borough_Toronto_data.head()


/home/dvh/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Create a map of Toronto with neighborhoods superimposed on top.

In [6]:
# create map of Manhattan using latitude and longitude values
map_borough_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(borough_Toronto_data['Latitude'], borough_Toronto_data['Longitude'], borough_Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_borough_Toronto)  
    
map_borough_Toronto

### Define Foursquare Credentials and Version

In [160]:
CLIENT_ID = 'VI1HZZN4HJQHMPBKYZUQ52OQVJFP3LLJ03ITAIQXTMEC3S1M' # your Foursquare ID
CLIENT_SECRET = 'EJ2C0NQ31QONYYFULJLYDIOV4NB22WI1TK4XDUQEXFKVVM2V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
# type your answer here



Your credentails:
CLIENT_ID: VI1HZZN4HJQHMPBKYZUQ52OQVJFP3LLJ03ITAIQXTMEC3S1M
CLIENT_SECRET:EJ2C0NQ31QONYYFULJLYDIOV4NB22WI1TK4XDUQEXFKVVM2V


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)        

In [9]:
# type your answer here

Toronto_venues = getNearbyVenues(names=borough_Toronto_data['Neighbourhood'],
                                   latitudes=borough_Toronto_data['Latitude'],
                                   longitudes=borough_Toronto_data['Longitude']
                                  )


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [12]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.06
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4              Gym  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2  Farmers Market  0.04
3      Restaurant  0.04
4          Bakery  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0     Coffee Shop  0.14
1  Breakfast Spot  0.09
2            Café  0.09
3   Burrito Place  0.05
4         Stadium  0.05


----Business reply mail Processing Centre969 Eastern----
              venue  freq
0               Spa  0.06
1     Auto Workshop  0.06
2        Comic Shop  0.06
3  Recording Studio  0.06
4        Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1  Airport Terminal  0.14
2   Airport Service  0.14
3             Pl

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Gym,Thai Restaurant,American Restaurant,Bar,Restaurant,Hotel,Japanese Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Bakery,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Grocery Store,Italian Restaurant,Performing Arts Venue,Pet Store,Office,Gym,Nightclub
3,Business reply mail Processing Centre969 Eastern,Comic Shop,Butcher,Skate Park,Spa,Farmers Market,Light Rail Station,Fast Food Restaurant,Brewery,Burrito Place,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina,Plane
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Pizza Place,Chinese Restaurant,Bakery,Convenience Store,Italian Restaurant,Indian Restaurant,Pub
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Bar,Spa
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bakery,Dumpling Restaurant,Coffee Shop,Hotpot Restaurant
8,Christie,Grocery Store,Café,Park,Convenience Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub,Coffee Shop
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Pub,Gastropub,Café,Bubble Tea Shop


## Run k-means to cluster the neighborhood 

In [15]:
# set number of clusters
kclusters = 8

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)


In [16]:
Toronto_merged = borough_Toronto_data

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Music Venue,Dance Studio,Neighborhood,Pub,Coffee Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Café,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Sandwich Place,Pet Store,Steakhouse,Food & Drink Shop,Sushi Restaurant,Ice Cream Shop,Pub,Fish & Chips Shop,Hotel,Burrito Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,Gastropub,American Restaurant,Fish Market,Juice Bar,New American Restaurant,Bookstore
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Bus Line,Park,Swim School,Dim Sum Restaurant,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [98]:
print(len(Toronto_merged),len(Toronto_grouped))dd

38 38


In [17]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now, let's define a query to search for Asian food that is within 5000 metres from the Toronto center.

In [99]:
radius = 10000
LIMIT = 500 # limit of number of venues returned by Foursquare API
print(search_query + ' .... OK!')

restaurant .... OK!


In [161]:
search_query = 'restaurant'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bba29d86a6071462c7dd0fd'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/winery_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d123941735',
      'name': 'Wine Bar',
      'pluralName': 'Wine Bars',
      'primary': True,
      'shortName': 'Wine Bar'}],
    'hasPerk': False,
    'id': '4ad4c05cf964a520dff520e3',
    'location': {'address': 'CN Tower',
     'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'crossStreet': '301 Front St. W',
     'distance': 1271,
     'formattedAddress': ['CN Tower (301 Front St. W)',
      'Toronto ON M5V 2T6',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.642537317144566,
       'lng': -79.38704201569328}],
     'lat': 43.642537317144566,
     'lng': -79.38704201569328,
     'postalCode': 'M5V 2T6',
     'state': 'ON'},
    'name': '360 Restaurant',
    'referralId': 'v-1538927065'},
   {'categori

In [102]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
print(dataframe.shape)
dataframe.head()

(50, 19)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",False,4ad4c05cf964a520dff520e3,CN Tower,CA,Toronto,Canada,301 Front St. W,1271,"[CN Tower (301 Front St. W), Toronto ON M5V 2T...","[{'label': 'display', 'lat': 43.64253731714456...",43.642537,-79.387042,NaN,M5V 2T6,ON,360 Restaurant,v-1538918065,NaN
1,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4ad4c05ff964a52048f720e3,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,Hemispheres Restaurant & Bistro,v-1538918065,NaN
2,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4baa4b39f964a520ec5b3ae3,181 Bay St.,CA,Toronto,Canada,in Brookfield Place,1077,"[181 Bay St. (in Brookfield Place), Toronto ON...","[{'label': 'display', 'lat': 43.64723237089591...",43.647232,-79.377591,NaN,M5J 2T3,ON,Marché Restaurant,v-1538918065,59869170
3,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",False,4ad4c060f964a5207ff720e3,323 Spadina Ave.,CA,Toronto,Canada,at D'Arcy St.,922,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...","[{'label': 'display', 'lat': 43.65431754076345...",43.654318,-79.398650,NaN,M5T 2E9,ON,Rol San Restaurant 龍笙棧,v-1538918065,NaN
4,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4ad4c05cf964a52006f620e3,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,1011,[37 King Street East (at Le Meridien King Edwa...,"[{'label': 'display', 'lat': 43.64929834396347...",43.649298,-79.376431,NaN,M5C 1E9,ON,Victoria's Restaurant,v-1538918065,498556908


In [103]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,360 Restaurant,Wine Bar,CN Tower,CA,Toronto,Canada,301 Front St. W,1271,"[CN Tower (301 Front St. W), Toronto ON M5V 2T...","[{'label': 'display', 'lat': 43.64253731714456...",43.642537,-79.387042,NaN,M5V 2T6,ON,4ad4c05cf964a520dff520e3
1,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,4ad4c05ff964a52048f720e3
2,Marché Restaurant,Restaurant,181 Bay St.,CA,Toronto,Canada,in Brookfield Place,1077,"[181 Bay St. (in Brookfield Place), Toronto ON...","[{'label': 'display', 'lat': 43.64723237089591...",43.647232,-79.377591,NaN,M5J 2T3,ON,4baa4b39f964a520ec5b3ae3
3,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,CA,Toronto,Canada,at D'Arcy St.,922,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...","[{'label': 'display', 'lat': 43.65431754076345...",43.654318,-79.398650,NaN,M5T 2E9,ON,4ad4c060f964a5207ff720e3
4,Victoria's Restaurant,Restaurant,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,1011,[37 King Street East (at Le Meridien King Edwa...,"[{'label': 'display', 'lat': 43.64929834396347...",43.649298,-79.376431,NaN,M5C 1E9,ON,4ad4c05cf964a52006f620e3
5,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,CA,Toronto,Canada,btwn Dundas St. W & D'Arcy St.,896,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,"[{'label': 'display', 'lat': 43.65386562507761...",43.653866,-79.398334,NaN,M5T 2E6,ON,4ae29812f964a520288f21e3
6,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,CA,Toronto,Canada,at Dundas St. W.,892,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...","[{'label': 'display', 'lat': 43.65278331265585...",43.652783,-79.398174,NaN,NaN,ON,4b072e9df964a52009f922e3
7,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,CA,Toronto,Canada,at Willison Sq,892,"[266 Spadina Ave (at Willison Sq), Toronto ON ...","[{'label': 'display', 'lat': 43.6522783893466,...",43.652278,-79.398039,NaN,M5T 2E4,ON,4b266f05f964a520657b24e3
8,The Hot House Restaurant & Bar,American Restaurant,35 Church St,CA,Toronto,Canada,at Front St E,1233,"[35 Church St (at Front St E), Toronto ON M5E ...","[{'label': 'display', 'lat': 43.64868288754344...",43.648683,-79.373741,NaN,M5E 1T3,ON,4ada5d5bf964a520e92121e3
9,Azure Restaurant & Bar,Restaurant,225 Frnt St W,CA,Toronto,Canada,NaN,1039,"[225 Frnt St W, Toronto ON M5V 2X3, Canada]","[{'label': 'display', 'lat': 43.64474919591934...",43.644749,-79.385113,Entertainment District,M5V 2X3,ON,4b223f5af964a520ba4424e3


In [78]:
dataframe_filtered.name

0                                        360 Restaurant
1                       Hemispheres Restaurant & Bistro
2                                     Marché Restaurant
3                                Rol San Restaurant 龍笙棧
4                                 Victoria's Restaurant
5                               Swatow Restaurant 汕頭小食家
6                    Sky Dragon Chinese Restaurant 龍翔酒樓
7                        Goldstone Noodle Restaurant 金石
8                                Azure Restaurant & Bar
9                        The Hot House Restaurant & Bar
10                                    Tundra Restaurant
11                         Aroma Fine Indian Restaurant
12                             Ka Chi Korean Restaurant
13                             New Sky Restaurant 小沙田食家
14                  Richtree Natural Market Restaurants
15                                ONE Restaurant/Lounge
16       Sassafraz | Cafe | Restaurant | Private Events
17                       Insomnia Restaurant and

### Let's explore some Asean restaurant

In [168]:
asean_id = [3,5,6,7,11,12,13,24,26,28,29,36,41,43,45,46,47,48]
len(asean_id)

18

In [152]:
Nr = len(asean_id)

rating_all = []
likes_all = []
tip_all = []
price_all = []


for i in range(Nr):
    venue_id = dataframe.id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    rating_all.append(result['response']['venue']['rating'])
    likes_all.append(result['response']['venue']['likes']['count'])
    tip_all.append(result['response']['venue']['stats']['tipCount'])
    price_all.append(result['response']['venue']['price']['message'])


KeyError: 'venue'

In [167]:
 # _Kyoto House Japanese Restaurant_ --- id=4ae4b055f964a520229d21e3
venue_id = dataframe.id[3]
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()


print(result['response']['venue']['rating'])
print(result['response']['venue']['likes']['count'])
print(result['response']['venue']['stats']['tipCount'])
print(result['response']['venue']['price']['message'])
# print(result['response']['venue']['rating'])

KeyError: 'venue'

In [163]:
result

{'meta': {'code': 429,
  'errorDetail': 'Quota exceeded',
  'errorType': 'quota_exceeded',
  'requestId': '5bba29e49fb6b75291787df0'},
 'response': {}}

7.6
30
10
Cheap


In [106]:
# create map of Manhattan using latitude and longitude values
map_restaurant_Toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_restaurant_Toronto)  
    
map_restaurant_Toronto

In [107]:
user_id = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])